In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
submission_test = pd.read_csv("/kaggle/input/llm-classification-finetuning/sample_submission.csv")
submission_test

In [ ]:
df_train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
df_train

In [ ]:
df_test = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
df_test

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

In [ ]:
X = df_train.drop(['model_a', 'model_b', 'winner_model_a', 'winner_model_b', 'winner_tie'], axis = 1)
X

In [ ]:
y = df_train[['winner_model_a', 'winner_model_b', 'winner_tie']].values

y = np.argmax(y, axis=1)
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
catagorical_feature = [col for col in X.columns if X[col].dtype == 'object']
catagorical_feature

In [ ]:
preprocessing = ColumnTransformer([
    ("prompt_tfidf", TfidfVectorizer(max_features=5000), "prompt"),
    ("resp_a_tfidf", TfidfVectorizer(max_features=5000), "response_a"),
    ("resp_b_tfidf", TfidfVectorizer(max_features=5000), "response_b"),
    ("num", "passthrough", ["id"])
])

In [ ]:
preprocessing

In [ ]:
feature_selection = SelectKBest(score_func = chi2, k=6)

In [ ]:
model = XGBClassifier(
    objective="multi:softprob",  
    num_class=3,                  
    eval_metric="mlogloss",       
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

In [ ]:
my_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("feature_selection", feature_selection),
    ("model", model)
])
my_pipeline

In [ ]:
scores = cross_val_score(my_pipeline, X_train, y_train, cv=5, scoring="accuracy")
print("Accuracy:", scores.mean())

In [ ]:
my_pipeline.fit(X_train, y_train)

In [ ]:
probs = my_pipeline.predict_proba(df_test)

submission = pd.DataFrame({
    "id": df_test["id"],
    "winner_model_a": probs[:,0],
    "winner_model_b": probs[:,1],
    "winner_tie": probs[:,2],
})
print(submission)
submission.to_csv("submission.csv", index=False)